<a href="https://colab.research.google.com/github/savankansagara1/sentiment-analysis-app/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [40]:
df=pd.read_csv('train.txt',sep=";",header=None,names=["text","emotion"])

In [41]:
df.isnull().sum()

,0
text,0
emotion,0


In [42]:
df.head()

,text,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [43]:
unique_emotions = df['emotion'].unique()
unique_emotions

array(['sadness', 'anger', 'love', 'surprise', 'fear', 'joy'],
      dtype=object)

In [44]:
emotion_numbers = {}
i = 0
for emo in unique_emotions:
  emotion_numbers[emo] = i
  i +=1

df['emotion'] = df['emotion'].map(emotion_numbers)

In [45]:
df

,text,emotion
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,1
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,1
...,...,...
15995,i just had a very brief time in the beanbag an...,0
15996,i am now turning and i feel pathetic that i am...,0
15997,i feel strong and good overall,5
15998,i feel like this was such a rude comment and i...,1


In [46]:
df["text"] = df["text"].apply(lambda x : x.lower())

In [47]:
import string
def remove_punc(txt):
  return txt.translate(str.maketrans('','',string.punctuation))

In [48]:
df["text"]=df["text"].apply(remove_punc)

In [49]:
def remove_numbers(txt):
  new = ""
  for i in txt:
    if not i.isdigit():
      new =new+i
  return new

df["text"]=df["text"].apply(remove_numbers)

In [50]:
def remove_emojis(txt):
  new = ""
  for i in txt:
    if i.isascii():
      new+=i
  return new # Moved the return statement outside the loop

df["text"] = df["text"].apply(remove_emojis)

In [51]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [52]:
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [53]:
stop_words=set(stopwords.words("english"))

In [54]:
df.loc[1]["text"]

'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake'

In [55]:
def remove(txt):
  words =word_tokenize(txt)
  cleaned = []
  for i in words:
    if not i in stop_words:
      cleaned.append(i)
  return ' '.join(cleaned)

In [56]:
df["text"]=df["text"].apply(remove)

In [57]:
df.loc[1]["text"]

'go feeling hopeless damned hopeful around someone cares awake'

In [58]:
df.head()

,text,emotion
0,didnt feel humiliated,0
1,go feeling hopeless damned hopeful around some...,0
2,im grabbing minute post feel greedy wrong,1
3,ever feeling nostalgic fireplace know still pr...,2
4,feeling grouchy,1


In [59]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df["text"], df["emotion"], test_size=0.20, random_state=42)

In [60]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score , classification_report

bow_vectorizer = CountVectorizer()
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)


nb_model = MultinomialNB()
nb_model.fit(X_train_bow, y_train)


pred_bow = nb_model.predict(X_test_bow)
print(accuracy_score(y_test, pred_bow))

0.7678125


In [61]:

pred_bow

array([0, 5, 0, ..., 5, 5, 0])

In [62]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


nb2_model = MultinomialNB()
nb2_model.fit(X_train_tfidf,y_train)

MultinomialNB()

In [63]:
y_pred = nb2_model.predict(X_test_tfidf)

In [64]:
print(accuracy_score(y_test, y_pred))

0.6609375


In [65]:
from sklearn.linear_model import LogisticRegression

In [66]:
logistic_model = LogisticRegression(max_iter=1000)

In [67]:

logistic_model.fit(X_train_tfidf,y_train)

LogisticRegression(max_iter=1000)

In [68]:
log_pred = logistic_model.predict(X_test_tfidf)

In [69]:

print(accuracy_score(y_test,log_pred ))

0.8615625


In [70]:
from sklearn.svm import SVC

In [71]:
svm_model = SVC(kernel='linear')

In [72]:
svm_model.fit(X_train_tfidf, y_train)

SVC(kernel='linear')

In [73]:
y_pred = svm_model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.883125


In [76]:
import joblib

# Save the model and vectorizer first
joblib.dump(svm_model, 'svm_model.pkl')
joblib.dump(tfidf_vectorizer, "vectorizer.pkl")

# Now load the model and vectorizer
with open("svm_model.pkl", "rb") as model_file:
    model = pickle.load(model_file)

with open("vectorizer.pkl", "rb") as vec_file:
    vectorizer = pickle.load(vec_file)

UnpicklingError: invalid load key, '\x02'.